In [1]:
import os
import pandas as pd
import numpy as np
import datetime

def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
os.getcwd()


'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Post_YoY'

In [2]:
weeks_2019Q2=[datetime.date(2019,5,11)+datetime.timedelta(days=x*7) for x in range(13)]

folder_2019_by_week="/home/jian/BigLots/2019_by_weeks/"

Q2_files_2019_all_types=list(recursive_file_gen(folder_2019_by_week))
Q2_files_2019_all_types=[x for x in Q2_files_2019_all_types if "aily" in x]
Q2_files_2019=[]
for week_end_date in weeks_2019Q2:
    file_path=[x for x in Q2_files_2019_all_types if str(week_end_date) in x]
    Q2_files_2019=Q2_files_2019+file_path
    
print(len(Q2_files_2019))
Q2_files_2019=sorted(Q2_files_2019)
Q2_files_2019

13


['/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-11/MediaStormDailySales20190514-121223-405.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-18/MediaStormDailySales20190521-111232-391.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-05-25/MediaStormDailySales20190528-111211-863.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-01/MediaStormDailySales20190604-111936-950.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-08/MediaStormDailySales20190611-122013-442.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-15/MediaStormDailySales20190618-112500-846.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-22/MediaStormDailySales20190625-111927-257.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-06-29/MediaStormDailySales20190702-113523-928.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-07-06/MediaStormDailySales20190709-115715-409.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-07-13/MediaStormDailyS

In [3]:
df_outptu_by_store_week=pd.DataFrame()
for file in Q2_files_2019:
    week_end_dt=file.split("by_weeks/MediaStorm_")[1][:10]
    df=pd.read_table(file,sep="|",dtype=str,usecols=["location_id","transaction_dt",'transaction_id','customer_id_hashed','item_transaction_amt'])
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    
    df_rewards=df[pd.notnull(df['customer_id_hashed'])]
    df_nonrewards=df[pd.isnull(df['customer_id_hashed'])]
    
    df_rewards_sales=df_rewards.groupby(["location_id"])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"rewards_sales"})
    df_nonrewards_sales=df_nonrewards.groupby(["location_id"])['item_transaction_amt'].sum().reset_index().rename(columns={"item_transaction_amt":"non_rewards_sales"})
    
    df_rewards_trans=df_rewards[["location_id","transaction_dt",'transaction_id','customer_id_hashed']].drop_duplicates()
    df_rewards_trans=df_rewards_trans.groupby("location_id")['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"rewards_trans"})
    
    df_nonrewards_trans=df_nonrewards[["location_id","transaction_dt",'transaction_id']].drop_duplicates()
    df_nonrewards_trans=df_nonrewards_trans.groupby("location_id")['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"non_rewards_trans"})

    df=pd.merge(df_rewards_sales,df_nonrewards_sales,on="location_id",how="outer")
    df=pd.merge(df,df_rewards_trans,on="location_id",how="outer")
    df=pd.merge(df,df_nonrewards_trans,on="location_id",how="outer")
    df=df.fillna(0)
    
    df['week_end_dt']=week_end_dt
    df_outptu_by_store_week=df_outptu_by_store_week.append(df)
    
df_outptu_by_store_week['Online/Instore']=np.where(df_outptu_by_store_week['location_id']=="6990","Online","Instore")

In [4]:
df_outptu_by_Type_week=df_outptu_by_store_week.groupby(["week_end_dt",'Online/Instore'])['rewards_sales','non_rewards_sales','rewards_trans','non_rewards_trans'].sum().reset_index()
df_outptu_by_Type_week=df_outptu_by_Type_week.sort_values(['Online/Instore',"week_end_dt"])


In [5]:
writer=pd.ExcelWriter("./BL_2019Q2_sales_rewards_level_all_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_outptu_by_Type_week.to_excel(writer,"total_by_week_type",index=False)
df_outptu_by_store_week.to_excel(writer,"total_by_store_week",index=False)
writer.save()

In [6]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Post_YoY'